# Regresión lineal
En el siguiente archivo se va a desarrollar la regresión lineal para las combinaciones de cada una de las variables que se encuentran en los datos provistos, agrupada por departamentos. Los datos se pueden encontrar [acá](https://docs.google.com/spreadsheets/u/1/d/12h1Pk1ZO-BDcGldzKW-IA9VMkU9RlUOPopFoOK6stdU/pubhtml).

### Imports

In [22]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import statsmodels.formula.api as smf

### Read file

In [23]:
municipios = pd.read_csv("/Users/Meili/Dropbox/Uniandes/Noveno/Visual/BonoParcial/Plebiscito-Colombia-2016/docs/Plebiscito.csv")
municipios.head()

# Create variables
variables=np.array(municipios.keys())
delete=np.array(['Municipio','Departamento','GanadorPrimeraVuelta','Ganador','AfectadoPorElConflictoPares','ZonasDeConcentracion','CultivosIlicitos','VotosPorElNo','PorcentajeNo','VotosPorElSi','PorcentajeSi','VotosValidos','VotosTotales','CuantosSalieronAVotar','Abstencion'])
variables=np.setdiff1d(variables,delete)

comparacion=np.array(['PorcentajeNo','PorcentajeSi','Abstencion'])

departamentos=municipios.Departamento.drop_duplicates()

# To numeric 

for i in range(7, len(variables)): 
    pd.to_numeric(municipios[variables[i]])

for i in range(len(comparacion)):
    pd.to_numeric(municipios[comparacion[i]])
    

### Regression and Correlation

In [32]:
results = {}
for departamento in departamentos:
    correlaciones = municipios[(municipios.Departamento == departamento)].corr()
    for j in range(len(variables)): 
        for k in range(len(comparacion)):            
            variable = str(variables[j])
            comparador = str(comparacion[k])
            if (departamento is not None) and (variable is not None) and (comparador is not None) :
                # create a fitted model in one line
                lm = smf.ols(formula=comparador + " ~ " + variable, data=municipios[(municipios.Departamento == departamento)]).fit()
                results[departamento + " - " + comparador + " - " + variable] = [lm.params[1], lm.params[0],correlaciones[comparador][variable]]

### Write results

In [33]:
import csv
with open('/Users/Meili/Dropbox/Uniandes/Noveno/Visual/BonoParcial/Plebiscito-Colombia-2016/docs/departamentos_results.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',quotechar=' ', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['Departamento', 'Variable1', 'Variable2', 'pendiente', 'intercepto', 'pearson'])
    for item in results:
        line = []
        line.append((item.split(' - ')[0]).strip())
        line.append((item.split(' - ')[1]).strip())
        line.append((item.split(' - ')[2]).strip())        
        line.extend(results[item])
        spamwriter.writerow([','.join(map(str, line))])